# WITHOUT recurrence

## NO non-linear iteration scheme implemented

## Independence of hrad and hinf > calculated BEFORE

In [ ]:
for time in range(0,profile.shape[0]):
    for depth in range(profile.shape[1]):
        depth_m=round(depth*node_spacing,2)
        if depth_m==0:
            profile[time,depth]=(2*time_step)/(density[0]*spec_heat_capacity[0]*node_spacing) * \
                                (solar_view_factor*(1-surface_albedo)*solar_rad_interp[time] + \
                                 convective_heat_air[time]*(temp_atm_interp[time]-profile[time-1,depth]) + \
                                 hrad[time]*(temp_sky_interp[time]-profile[time-1,depth]) + \
                                 (thermal_conductivity[0]/time_step)*(profile[time-1,depth+1]-profile[time-1,depth])) + \
                                profile[time-1,depth]
            
        elif depth_m<=pavement_depth[-1]:
            for interface in range(len(pavement_depth)):
                if depth_m<pavement_depth[interface]:
                    profile[time,depth] = (time_step)/(density[interface]*spec_heat_capacity[interface]*node_spacing) * \
                                          (thermal_conductivity[interface]/node_spacing * \
                                          (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                          profile[time-1,depth]
                    break
                    
                elif depth_m==pavement_depth[interface]:
                    profile[time,depth]=0.5*((2*node_spacing*thermal_conductivity[interface]+ \
                                              thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                              profile[time,depth-1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                            +(2*node_spacing*thermal_conductivity[interface+1]+ \
                                              thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                              profile[time,depth+1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                            )
                    if time==1: print('interface nº'+str(interface))
                    break
                    
        elif depth_m!=ground_layer_depth:
            profile[time,depth] = (time_step)/(density[-1]*spec_heat_capacity[-1]*node_spacing) * \
                                          (thermal_conductivity[-1]/node_spacing * \
                                          (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                          profile[time-1,depth]
            
        else:
            profile[time,depth] = deep_ground_temp
            if time==1: 
                print('deep ground temp reached')
                print('depth =',depth_m)


In [ ]:
np.linalg.norm(profile,'fro')

# WITH recurrence

## Non-linear iteration scheme implemented

## hrad and hinf must be calculated within the scheme

Here, we calculate hrad and hinf on a **global** time-scale.  

-- hrad and hinf are actualized **at every nlin iteration**, but not on a time basis...

In [ ]:
for ite in range(1):
    
    if ite==0: 
        profile=np.zeros((len(time_interp),int(ground_layer_depth/node_spacing)+1))
        #profile[0,:].fill(deep_ground_temp)
        #profile[-1,:].fill(deep_ground_temp)
        profile.fill(deep_ground_temp)
    else:
        profile[0,:]=profile[-1,:]
        #profile[-1,:]=profile[0,:]
        profile_old=profile
        
    hrad = sky_view_factor*surface_emissivity*stefan_boltzmann*(profile[:,0]**2+temp_sky_interp**2)*(profile[:,0]+temp_sky_interp)

    film_temp = 0.5*(profile[:,0]+temp_atm_interp)
    film_viscosity = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                     *(ref_viscosity_flow[1]-ref_viscosity_flow[0])+ref_viscosity_flow[0]
    film_conductivity = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                        *(ref_conductivity_flow[1]-ref_conductivity_flow[0])+ref_conductivity_flow[0]
    film_diffusivity = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                        *(ref_diffusivity_flow[1]-ref_diffusivity_flow[0])+ref_diffusivity_flow[0]
    film_prandtl = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                   *(ref_prandtl_flow[1]-ref_prandtl_flow[0])+ref_prandtl_flow[0]
    film_beta = 1 / film_temp

    film_reynolds = wind_speed_interp * characteristic_length / film_viscosity
    film_nusselt_laminar = 0.664 * film_reynolds**(0.5) * film_prandtl**(1/3)
    film_nusselt_turbulent = 0.037 * film_reynolds**(0.8) * film_prandtl**(1/3)
    film_nusselt_actual = np.zeros((len(film_reynolds)))

    info=0
    for i in range(len(film_reynolds)):
        if film_reynolds[i]<5e5:
            film_nusselt_actual[i]=film_nusselt_laminar[i]
        else:
            film_nusselt_actual[i]=film_nusselt_turbulent[i]
            if info==0: print('TURBULENT FLOW at ite = '+str(ite+1))
            info=1

    convective_heat_air = film_nusselt_actual * film_conductivity / characteristic_length

    delta_t=(node_spacing**2)*density[0]*spec_heat_capacity[0] / (2*(hrad*node_spacing+convective_heat_air*node_spacing+thermal_conductivity[0]))
    stability = delta_t > time_step
    print('mean_CFL = '+str(np.mean(delta_t)))
    check_stability = np.all(stability)
    if check_stability==False: 
        print('Unstable forward (explicit) scheme')
        break
        
    for time in range(0,profile.shape[0]):
    #for time in range(1):
        for depth in range(profile.shape[1]):
            depth_m=round(depth*node_spacing,2)
            if depth_m==0:
                profile[time,depth]=(2*time_step)/(density[0]*spec_heat_capacity[0]*node_spacing) * \
                                    (solar_view_factor*(1-surface_albedo)*solar_rad_interp[time] + \
                                     convective_heat_air[time]*(temp_atm_interp[time]-profile[time-1,0]) + \
                                     hrad[time]*(temp_sky_interp[time]-profile[time-1,0]) + \
                                     (thermal_conductivity[0]/time_step)*(profile[time-1,1]-profile[time-1,0])) + \
                                    profile[time-1,depth]

            elif depth_m<=pavement_depth[-1]:
                for interface in range(len(pavement_depth)):
                    if depth_m<pavement_depth[interface]:
                        profile[time,depth] = time_step/(density[interface]*spec_heat_capacity[interface]*node_spacing) * \
                                              ((thermal_conductivity[interface]/node_spacing) * \
                                              (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                              profile[time-1,depth]
                        break

                    elif depth_m==pavement_depth[interface]:
                        profile[time,depth]=0.5*((2*node_spacing*thermal_conductivity[interface]+ \
                                                  thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                                  profile[time,depth-1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                                +(2*node_spacing*thermal_conductivity[interface+1]+ \
                                                  thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                                  profile[time,depth+1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                                )
                        break

            elif depth_m!=ground_layer_depth:
                profile[time,depth] = (time_step)/(density[-1]*spec_heat_capacity[-1]*node_spacing) * \
                                              (thermal_conductivity[-1]/node_spacing * \
                                              (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                              profile[time-1,depth]

            else:
                profile[time,depth] = deep_ground_temp
    
    if ite!=0:
        profile_dif=profile_old-profile
        print(np.linalg.norm(profile,'fro'))

# Implementation with corrected surface equation

In [ ]:
profile = np.zeros((len(time_interp),int(ground_layer_depth/node_spacing)+1))

stab_flag = False

for ite in range(1):
    
    print('\n')
    print('##############')
    print('  nlin_ite'+str(ite+1))
    print('##############')
    
    if ite==0: 
        profile[-1,:].fill(deep_ground_temp)
    else:
        profile_old=profile
    
    #for time in range(0,profile.shape[0]):
    for time in range(0,5):
        
        hrad = sky_view_factor*surface_emissivity*stefan_boltzmann*(profile[time-1,0]**2+temp_sky_interp[time]**2)*(profile[time-1,0]+temp_sky_interp[time])
        
        
        print('\n')
        print('hrad =',hrad)
        print('------------')
        
        
        film_temp = 0.5*(profile[time-1,0]+temp_atm_interp[time])
        film_viscosity = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                         *(ref_viscosity_flow[1]-ref_viscosity_flow[0])+ref_viscosity_flow[0]
        film_conductivity = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                            *(ref_conductivity_flow[1]-ref_conductivity_flow[0])+ref_conductivity_flow[0]
        film_diffusivity = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                            *(ref_diffusivity_flow[1]-ref_diffusivity_flow[0])+ref_diffusivity_flow[0]
        film_prandtl = ((film_temp-ref_temp_flow[0])/(ref_temp_flow[1]-ref_temp_flow[0])) \
                       *(ref_prandtl_flow[1]-ref_prandtl_flow[0])+ref_prandtl_flow[0]
        film_beta = 1 / film_temp

        film_reynolds = wind_speed_interp[time] * characteristic_length / film_viscosity
        film_nusselt_laminar = 0.664 * film_reynolds**(0.5) * film_prandtl**(1/3)
        film_nusselt_turbulent = 0.037 * film_reynolds**(0.8) * film_prandtl**(1/3)

        if film_reynolds<5e5:
            film_nusselt_actual=film_nusselt_laminar
            print('Laminar flow')
        else:
            film_nusselt_actual=film_nusselt_turbulent
            print('Turbulent flow')

        convective_heat_air = film_nusselt_actual * film_conductivity / characteristic_length
        #print('hinf =',convective_heat_air)
        
        delta_t=(node_spacing**2)*density[0]*spec_heat_capacity[0] / (2*(hrad*node_spacing+convective_heat_air*node_spacing+thermal_conductivity[0]))
        stability = delta_t > time_step
        
        
        print('------------')
        print('CFL criterion =',delta_t)
        print('------------')
          
        
        if stability==False: 
            print('Unstable FD scheme, breaking!')
            stab_flag = True
            break
        
        ##### A / B / C surface coefficients #####
        
        delta_surf = (2*time_step)/(density[0]*spec_heat_capacity[0]*node_spacing)
        
        Asurf = 1-(delta_surf*(hrad+convective_heat_air+thermal_conductivity[0]/node_spacing))
        Bsurf = delta_surf*thermal_conductivity[0]/node_spacing
        Csurf = delta_surf*(hrad*temp_sky_interp[time]+convective_heat_air*temp_atm_interp[time] + \
                           solar_view_factor*(1-surface_albedo)*solar_rad_interp[time])
        
        print('delta_surf =',delta_surf)
        print('Asurf =',Asurf)
        print('Dsurf =',Bsurf)
        print('Csurf =',Csurf)
        print('------------')
        
        ###################################################        
        
        #for depth in range(profile.shape[1]):
        for depth in range(5):
            depth_m=round(depth*node_spacing,2)
            if depth_m==0:
                profile[time,depth]=(2*time_step)/(density[0]*spec_heat_capacity[0]*node_spacing) * \
                                    (solar_view_factor*(1-surface_albedo)*solar_rad_interp[time] + \
                                     convective_heat_air*(temp_atm_interp[time]-profile[time-1,depth]) + \
                                     hrad*(temp_sky_interp[time]-profile[time-1,depth]) + \
                                     (thermal_conductivity[0]/node_spacing)*(profile[time-1,depth+1]-profile[time-1,depth])) + \
                                    profile[time-1,depth]
                
            elif depth_m<=pavement_depth[-1]:
                for interface in range(len(pavement_depth)):
                    if depth_m<pavement_depth[interface]:
                        profile[time,depth] = (time_step)/(density[interface]*spec_heat_capacity[interface]*node_spacing) * \
                                              (thermal_conductivity[interface]/node_spacing * \
                                              (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                              profile[time-1,depth]
                        break

                    elif depth_m==pavement_depth[interface]:
                        profile[time,depth]=0.5*((2*node_spacing*thermal_conductivity[interface]+ \
                                                  thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                                  profile[time,depth-1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                                +(2*node_spacing*thermal_conductivity[interface+1]+ \
                                                  thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface])* \
                                                  profile[time,depth+1]/(thermal_conductivity[interface]*thermal_conductivity[interface+1]*interface_contact_res[interface]+node_spacing*thermal_conductivity[interface]+node_spacing*thermal_conductivity[interface+1]) \
                                                )
                        break

            elif depth_m!=ground_layer_depth:
                profile[time,depth] = (time_step)/(density[-1]*spec_heat_capacity[-1]*node_spacing) * \
                                              (thermal_conductivity[-1]/node_spacing * \
                                              (profile[time-1,depth-1]+profile[time-1,depth+1]-2*profile[time-1,depth])) + \
                                              profile[time-1,depth]

            else:
                profile[time,depth] = deep_ground_temp
                
            print('T('+str(time)+','+str(depth)+') =',profile[time,depth])
    
    if stab_flag == True:
        break
    
    if ite!=0:
        profile_dif=profile_old-profile
        print('\n')
        print('############################################')
        print('FROBENIUS MATRIX NORM Fnorm(X)')
        print('X = profile[nlin_ite'+str(ite)+'] - profile[nlin_ite'+str(ite+1)+']')
        print('Fnorm(X) = ',np.linalg.norm(profile,'fro'))
        print('############################################')